In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score # Для кросс валидации
from sklearn.ensemble import RandomForestClassifier # Случайный лес
from sklearn.model_selection import GridSearchCV # Перебор всех комбинаций гиперпараметра
from sklearn.metrics import roc_auc_score, make_scorer

In [2]:
df=pd.read_csv("../kaggle/Iris/Iris.csv")
cat_col=df.select_dtypes(include="object").columns
num_col=df.select_dtypes(include="number").columns
print(f"Numeric columns: {num_col}\nCategoric columns: {cat_col}\n")
df.drop(columns=['Id'],inplace=True)

Numeric columns: Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')
Categoric columns: Index(['Species'], dtype='object')



In [4]:
# df.nunique()
# for col in df.columns:
display(df.iloc[:,4].value_counts())

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [5]:
X=df.drop(columns=["Species"],inplace=False)
Y=df.iloc[:,4]
Y

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: Species, Length: 150, dtype: object

### Работаем с кросс-валидацией

In [ ]:
model=RandomForestClassifier()
scores=cross_val_score(model,X,Y,cv=5)
print(scores.mean())
model.fit(X,Y)
pred_test_data=model.predict_proba(X)
# print(pred_test_data)
model.predict(X)

0.9466666666666667


'Iris-setosa'

### Работа с GridSearhCV

In [17]:
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200]
}
scores=make_scorer(roc_auc_score,multi_class='ovo',needs_proba=True)


grid = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=-1,cv=5, scoring=scores)
grid.fit(X, Y)
print(f"{grid.best_score_} \n{grid.best_estimator_}")

/Users/edgar/.venvs/myenv/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


0.9933333333333334 
RandomForestClassifier(max_depth=3)


### Stecked generalization
**Как происходит**
* Разбиение на train/test
* Разбиение на k фолдов
* Базовая модель на k-1 фолде обучается, на k-ом прогноз
* Предыдущие шаги повторяются для каждой базовой модели
* Получабтся новые признаки состоящие из предсказаний прошлых моделей + y_train
* 